In [ ]:
!unzip /kaggle/input/covid19-pneumonia-and-normal-chest-xray-dataset/898720e7-9fcd-49f0-87ba-08c979e6f35e

In [38]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import time

In [ ]:
dir = Path("/kaggle/working/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset")

filepaths = list(dir.glob(r'**/*.*'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

In [ ]:
filepaths = pd.Series(filepaths, name='filepath').astype(str)
labels = pd.Series(labels, name='label')

df_big = pd.concat([filepaths , labels] , axis=1)
df_big.head()

In [ ]:
df_big.label.value_counts()

In [ ]:
def resize_images(df, save_folder, size=(224, 224), extenstion='png'):
    for i in range(len(df['filepath'])):
        path = df['filepath'][i]
        label = df['label'][i]
        file_name = path.split('/')[5].replace('.', '_')

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, size, interpolation=cv2.INTER_NEAREST)
        
        file_name = f"{save_folder}/{label}/{file_name}.{extenstion}"
        cv2.imwrite(file_name, img.astype(np.uint8))
        img2 = mpimg.imread(file_name)
        if i % 1000 == 0:
            print(i)
    print(i)

In [ ]:
SAVE_FOLDER = "resized_dataset"
SIZE = (224, 224)

for label in df_big.label.unique():
    os.makedirs(SAVE_FOLDER+'/'+label, exist_ok=True)
    
resize_images(df_big, SAVE_FOLDER, size=SIZE)

In [ ]:
def resize_with_ratio(df, save_folder, size=(224, 224), extenstion='png'):
    for i in range(len(df['filepath'])):
        path = df['filepath'][i]
        label = df['label'][i]
        file_name = path.split('/')[5].replace('.', '_')

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        # add zeros to the narrow parts of the image
        height = img.shape[0]
        width = img.shape[1]
        if width > height:
            add_up = (width - height) // 2
            add_bottom = width - height - add_up
            add_left = 0
            add_right = 0
            img = cv2.copyMakeBorder(img, add_up, add_bottom, add_left, add_right, cv2.BORDER_CONSTANT, None, value = 0)
        elif width < height:
            add_up = 0
            add_bottom = 0
            add_left = (height - width) // 2
            add_right = height - width - add_up
            img = cv2.copyMakeBorder(img, add_up, add_bottom, add_left, add_right, cv2.BORDER_CONSTANT, None, value = 0)
        
        img = cv2.resize(img, size, interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(f"{save_folder}/{label}/{file_name}.{extenstion}", img.astype(np.uint8))
        if i % 1000 == 0:
            print(i)

In [ ]:
SAVE_FOLDER = "resized_with_ratio"
SIZE = (224, 224)

for label in df_big.label.unique():
    os.makedirs(SAVE_FOLDER+'/'+label, exist_ok=True)
    
resize_with_ratio(df_big, SAVE_FOLDER, size=SIZE)

In [ ]:
!ls

In [ ]:
!rm -r COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset

> # Code added by Mehran

In [1]:

import pandas as pd 
import tensorflow as tf 
from tensorflow.keras import layers, Input, Sequential
from tensorflow.keras.layers import ReLU ,LeakyReLU, GlobalAveragePooling2D , Dropout
from tensorflow.keras.layers import Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import image_dataset_from_directory #### add 

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [2]:
def prepare_dataset(path, size=224, color_mode='grayscale', batch_size=32):
    seed = 1
    shuffle_value = True
    val_test_split = 0.16 + 0.2

    train_ds = image_dataset_from_directory(directory = path,
                labels='inferred',
                label_mode='categorical',
                image_size = (size, size),
                batch_size= batch_size,
                validation_split = val_test_split,
                subset = "training",
                seed = seed,
                color_mode = color_mode,
                shuffle = shuffle_value)

    val_ds = image_dataset_from_directory(directory = path,
                labels='inferred',
                label_mode='categorical',
                image_size = (size, size),
                batch_size = batch_size,
                validation_split = val_test_split,
                subset = "validation",
                seed = seed,
                color_mode = color_mode,
                shuffle = shuffle_value)
    val_batches = tf.cast(tf.data.experimental.cardinality(val_ds), dtype=tf.float64)
    vali_test_ratio = tf.cast( (tf.constant(0.2, dtype=tf.float64) * val_batches) // tf.constant(val_test_split, dtype=tf.float64) , tf.int64)
    test_ds = val_ds.take(vali_test_ratio)
    val_ds = val_ds.skip(vali_test_ratio)


    return train_ds, val_ds, test_ds


def get_pred_true_labels(model, generator):
    """The function take the keras model instance and the generator and returns
    y_true the true class values
    y_pred the predicted class values
    
    The function set the shuffling of a generator to false, 
    so to turn on shuffling the generator should be reinitialized
    """
    generator.shuffle = False
    generator.index_array = None
    y_pred = model.predict(generator)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.array(generator.classes)
    return y_true, y_pred

def plot_confusion_matrix(cm, title):
    with sns.axes_style("white"):
        disp = ConfusionMatrixDisplay(confusion_matrix = cm, 
                                      display_labels = ['covid', 'normal', 'pneumonia'],)
        fig, ax = plt.subplots(figsize=(8, 6))
        disp.plot(cmap=plt.cm.Blues, ax=ax)
        plt.title(title);

def report_to_df(rep_dict):
    df = pd.DataFrame(rep_dict).reset_index()
    df = df.rename(columns={"index": "metrics"})
    df = df.drop(['accuracy'], axis=1)
    df = pd.melt(df, id_vars=['metrics'], 
        value_vars=['covid', 'normal', 'pneumonia', 'macro avg', 'weighted avg'],
        var_name='label', value_name='value')
    return df[(df.metrics != 'support')]

In [3]:
size=224
color_mode='grayscale'  
batch_size=32

In [4]:
path = "/kaggle/working/resized_with_ratio"
train_ds, val_ds, test_ds = prepare_dataset(path, size, color_mode, batch_size)

Found 4575 files belonging to 3 classes.
Using 2928 files for training.


2023-02-11 18:48:14.951484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 18:48:14.966279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 18:48:14.967459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 18:48:14.970013: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 4575 files belonging to 3 classes.
Using 1647 files for validation.


In [5]:
train_ds = train_ds.cache()
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
val_ds = val_ds.cache()
val_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
# ###run for the first time to avoid 'Cleanup called...' message during the training
# for x , y in train_ds:
#     pass

# for x , y in val_ds:
#     pass

# for x , y in test_ds:
#     pass

In [10]:
def conv_bn(in_channels, out_channels, kernel_size, strides, padding, groups=1):
    blk = Sequential()
    blk.add(ZeroPadding2D(padding=(padding, padding)))
    blk.add(
            Conv2D(out_channels, kernel_size=kernel_size, strides=strides,
                   padding='valid',  
                   groups=groups, 
                   use_bias=False))
    blk.add(BatchNormalization())    
    return blk

In [11]:
class RepVGGBlock(tf.keras.Model):

    def __init__(self, in_channels, out_channels, kernel_size = 3,
                 strides=1, padding= 1, dilation=1, groups=1, 
                 deploy=False,
                 nonlinearity_leakyrelu= False):
        super(RepVGGBlock, self).__init__()
        self.deploy = deploy 
        self.groups = groups
        self.in_channels = in_channels
        self.padding = padding
        self.zeropadding = ZeroPadding2D(padding=(self.padding, self.padding))
        
        assert kernel_size == 3
        assert padding == 1
    
        padding_11 = padding - kernel_size // 2

        if nonlinearity_leakyrelu:
            self.nonlinearity = LeakyReLU(alpha=0.3)
        else: 
            self.nonlinearity = ReLU()
        
        if deploy:
            self.rbr_reparam = Conv2D(filters=out_channels, kernel_size=kernel_size, strides=strides,
                                      padding= 'valid', dilation_rate=dilation, groups=groups, use_bias=True)
        else:
            self.rbr_identity = BatchNormalization() if out_channels == in_channels and strides == 1 else None
            self.rbr_dense = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, strides=strides, padding=padding, groups=groups)
            self.rbr_1x1 = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=1, strides=strides, padding=padding_11, groups=groups)
            print('RepVGG Block, identity = ', self.rbr_identity)
            
            
    def call(self, inputs):
        if hasattr(self, 'rbr_reparam'):
            
            return self.nonlinearity(self.rbr_reparam(self.zeropadding(inputs)))
            
        if self.rbr_identity is None:
            id_out = 0
        else:
            id_out = self.rbr_identity(inputs)

        return self.nonlinearity(self.rbr_dense(inputs) + self.rbr_1x1(inputs) + id_out)
    

    def get_equivalent_kernel_bias(self):
        kernel3x3, bias3x3 = self._fuse_bn_tensor(self.rbr_dense)
        kernel1x1, bias1x1 = self._fuse_bn_tensor(self.rbr_1x1)
        kernelid, biasid = self._fuse_bn_tensor(self.rbr_identity)
        return kernel3x3 + self._pad_1x1_to_3x3_tensor(kernel1x1) + kernelid, bias3x3 + bias1x1 + biasid
    
    
    def _pad_1x1_to_3x3_tensor(self, kernel1x1):
        if kernel1x1 is None:
            return 0
        else:
            paddings = tf.constant([[1,1],[1,1],[0,0],[0,0]])
            return tf.pad(kernel1x1, paddings, "CONSTANT")
        
        
    def _fuse_bn_tensor(self, branch):
        if branch is None:
            return 0, 0
        if isinstance(branch, Sequential):
            kernel = branch.layers[1].get_weights()[0]
            running_mean = branch.layers[2].moving_mean 
            running_var = branch.layers[2].moving_variance
            gamma = branch.layers[2].gamma
            beta = branch.layers[2].beta
            eps = branch.layers[2].epsilon
        else:
            assert isinstance(branch, BatchNormalization)
            if not hasattr(self, 'id_tensor'):
                input_dim = self.in_channels // self.groups
                kernel_value = np.zeros((3, 3, self.in_channels, input_dim), dtype=np.float32)
                for i in range(self.in_channels):
                    kernel_value[1,1, i, i % input_dim] = 1
                self.id_tensor = tf.convert_to_tensor(kernel_value)
            kernel = self.id_tensor
            running_mean = branch.moving_mean 
            running_var = branch.moving_variance
            gamma = branch.gamma
            beta = branch.beta
            eps = branch.epsilon
            
        std = tf.math.sqrt((running_var + eps))
        t = (gamma / std).reshape(1, 1, 1, -1)
        o1 = kernel * t
        o2 = beta - running_mean * gamma / std 
        return o1, o2 

    
    def switch_to_deploy(self):
        if hasattr(self, 'rbr_reparam'):
            return
        kernel, bias = self.get_equivalent_kernel_bias()
        self.rbr_reparam = Conv2D(self.rbr_dense.layers[1].filters, kernel_size=self.rbr_dense.layers[1].kernel_size,
                                   strides=self.rbr_dense.layers[1].strides,
                                   padding='valid', 
                                   groups=self.rbr_dense.layers[1].groups, 
                                   use_bias=True,
                                   weights=[kernel,bias])
        
        self.__delattr__('rbr_dense')
        self.__delattr__('rbr_1x1')
        if hasattr(self, 'rbr_identity'):
            self.__delattr__('rbr_identity')
        if hasattr(self, 'id_tensor'):
            self.__delattr__('id_tensor')
        self.deploy = True

In [12]:
class RepVGG(tf.keras.Model):

    def __init__(self, stage_inplanes, num_blocks, in_channels=1, num_classes=2, dropout = 0.1, name = 'RepVGG', deploy=False, use_checkpoint=False):
        super(RepVGG, self).__init__(name= name)
        self.in_channels = in_channels
        self.deploy = deploy
        self.use_checkpoint = use_checkpoint
        
        self.inplanes_0 = stage_inplanes [0]
        self.inplanes_1 = stage_inplanes [1]
        self.inplanes_2 = stage_inplanes [2]
        self.inplanes_3 = stage_inplanes [3]
        self.inplanes_4 = stage_inplanes [4]
        
        self.in_planes = self.inplanes_0                    
        self.stage0 = RepVGGBlock(in_channels=self.in_channels, out_channels=self.inplanes_0, kernel_size=3, strides=1, padding=1, deploy=self.deploy)
        self.cur_layer_idx = 1
        self.dropout = Dropout(dropout) 
        
        self.stage1 = self._make_stage(self.inplanes_1, num_blocks[0], strides=1)
        self.stage2 = self._make_stage(self.inplanes_2, num_blocks[1], strides=1)
        self.stage3 = self._make_stage(self.inplanes_3, num_blocks[2], strides=2)
        self.stage4 = self._make_stage(self.inplanes_4, num_blocks[3], strides=2)
        
        self.gap = GlobalAveragePooling2D(keepdims=False)
        self.linear = Dense(num_classes, activation= 'softmax', name='fc')

    def _make_stage(self, planes, num_blocks, strides):
        strideslist = [strides] + [1]*(num_blocks-1)
        blocks = []
        for stride in strideslist:
            cur_groups = 1
            blocks.append(RepVGGBlock(in_channels=self.in_planes, out_channels=planes, kernel_size=3,
                                      strides=stride, padding=1, groups=cur_groups, deploy=self.deploy))
            self.in_planes = planes
            self.cur_layer_idx += 1
        return blocks

    def switch_to_deploy(self):
        self.stage0.switch_to_deploy()
        for stage in (self.stage1, self.stage2, self.stage3, self.stage4):
            for block in stage:
                block.switch_to_deploy()  
        
    def call(self, x):
        x = layers.Rescaling(1./255)(x)
        out = self.stage0(x)
        for stage in (self.stage1, self.stage2, self.stage3, self.stage4):
            for block in stage:
                if self.use_checkpoint:
                    out = checkpoint.checkpoint(block, out)
                else:
                    out = block(out)
        out = self.gap(out)
        out = Flatten()(out)
        out = self.dropout(out)
        out = self.linear(out)
        return out

In [13]:
def train_model(model, model_name, learning_rate= 0.0001,epochs= 60):


    model.compile(loss="categorical_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    metrics=['accuracy',
                             'Precision',
                             'Recall',
                             tf.keras.metrics.AUC(
                                    num_thresholds=200,
                                    curve="ROC",
                                    summation_method="interpolation",
                                    multi_label=False,
                                    from_logits=False)])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,),
        tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{model_name}.h5",
        save_best_only=True,
        save_weights_only=True,
        monitor="val_accuracy")
        ]
    
    history = model.fit(
            train_ds,
            epochs= 60,
            callbacks=callbacks,
            verbose=1,
            validation_data=val_ds)
    
    return history



def load_model_weights(model, size=224, channels=1, path = '/kaggle/input/repvgg-7-layers/repvgg-7-layers.h5'):
    model.build((None, size,size, channels)) 
    model.load_weights(path)
    model.compile(
            optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss= tf.keras.losses.CategoricalCrossentropy(from_logits= False), #'categorical_crossentropy',
            metrics=['accuracy',
                     'Precision',
                     'Recall',
                     tf.keras.metrics.AUC(
                                            num_thresholds=200,
                                            curve="ROC",
                                            summation_method="interpolation",
                                            multi_label=False,
                                            from_logits=False)])

In [20]:
model_name = "repvgg-7-layers"
repvgg_model = RepVGG(stage_inplanes= [64, 64, 128, 128, 128], num_blocks=[1, 2, 2, 1], in_channels=1, num_classes=3, name= model_name,
                 deploy=False, use_checkpoint=False)

RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f031b177a90>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7efed5a1cf50>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7efed590a350>
RepVGG Block, identity =  None


In [21]:
# repvgg_model.build((None,size,size,1))
# repvgg_model.summary()

In [22]:
# load_model_weights(model=repvgg_model, size= 224, channels=1, path= '/kaggle/input/repvgg-7-layers/repvgg-7-layers.h5')

In [23]:
repvgg_history = train_model(model= repvgg_model, model_name= model_name, learning_rate= 0.0001,epochs= 60)

Epoch 1/60


2023-02-11 18:51:14.385332: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


34/92 [==========>...................] - ETA: 36s - loss: 0.7670 - accuracy: 0.6746 - precision: 0.7367 - recall: 0.5478 - auc_1: 0.8405

2023-02-11 18:51:36.569472: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.45GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


36/92 [==========>...................] - ETA: 34s - loss: 0.7614 - accuracy: 0.6788 - precision: 0.7370 - recall: 0.5521 - auc_1: 0.8436

...

42/92 [============>.................] - ETA: 31s - loss: 0.7334 - accuracy: 0.6964 - precision: 0.7512 - recall: 0.5796 - auc_1: 0.8565

2023-02-11 18:51:41.709950: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.45GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


44/92 [=============>................] - ETA: 30s - loss: 0.7243 - accuracy: 0.7010 - precision: 0.7546 - recall: 0.5874 - auc_1: 0.8601

2023-02-11 18:51:42.997165: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.45GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


92/92 [==============================] - 72s 711ms/step - loss: 0.6179 - accuracy: 0.7623 - precision: 0.8043 - recall: 0.6906 - auc_1: 0.9024 - val_loss: 2.0277 - val_accuracy: 0.3395 - val_precision: 0.3395 - val_recall: 0.3395 - val_auc_1: 0.4972
Epoch 2/60
92/92 [==============================] - 61s 667ms/step - loss: 0.4388 - accuracy: 0.8528 - precision: 0.8687 - recall: 0.8364 - auc_1: 0.9499 - val_loss: 2.1209 - val_accuracy: 0.3395 - val_precision: 0.3395 - val_recall: 0.3395 - val_auc_1: 0.5027
Epoch 3/60
92/92 [==============================] - 61s 665ms/step - loss: 0.3733 - accuracy: 0.8801 - precision: 0.8941 - recall: 0.8648 - auc_1: 0.9629 - val_loss: 1.7784 - val_accuracy: 0.4021 - val_precision: 0.4039 - val_recall: 0.3888 - val_auc_1: 0.5246
Epoch 4/60
92/92 [==============================] - 61s 666ms/step - loss: 0.3419 - accuracy: 0.8897 - precision: 0.9017 - recall: 0.8805 - auc_1: 0.9687 - val_loss: 1.4794 - val_accuracy: 0.3302 - val_precision: 0.3257 - val_re

In [26]:
repvgg_history_df = pd.DataFrame(repvgg_history.history)
repvgg_history_df.to_csv('/kaggle/working/repvgg-7-layers-history.csv')

In [36]:
def evaluate_model(model= repvgg_model, train=True, validation=True, test=True, verbose=0):
    print("Evaluating with primary architechture\n")
    if train:
        start = time.time()
        preds = model.evaluate(train_ds, verbose=0)
        stop = time.time()
        
        print('Primary Architecture')
        print('Train Loss = {:.5f}'.format(preds[0]))
        print('Train Accuracy = {:.2f}%'.format(preds[1]*100))
        print(f'Time on train set with residual connections on GPU took: {(stop-start)/60} minutes')
        
    if validation:
        start = time.time()
        preds = model.evaluate(val_ds, verbose=0)
        stop = time.time()
        print()
        print('Primary Architecture')
        print('Validation Loss = {:.5f}'.format(preds[0]))
        print('Validation Accuracy = {:.2f}%'.format(preds[1]*100))
        print(f'Time on validation set with residual connections on GPU took: {(stop-start)/60} minutes')
        
    if test:
        start = time.time()
        preds = model.evaluate(test_ds, verbose=0)
        stop = time.time()
        print()
        print('Primary Architecture')
        print('Test Loss = {:.5f}'.format(preds[0]))
        print('Test Accuracy = {:.2f}%'.format(preds[1]*100))
        print(f'test time with residual connections on GPU took: {(stop-start)/60} minutes')

        
    print("Evaluating with secondary architechture", end= '/n')
    model.switch_to_deploy()

    if train:
        start = time.time()
        preds = model.evaluate(train_ds, verbose=0)
        stop = time.time()
        print()
        print('Secondary Architecture')
        print('Loss and Train Accuracy after switching to deploy mode:', end= '\n')
        print('Loss = {:.5f}'.format(preds[0]))
        print('Train Accuracy = {:.2f}%'.format(preds[1]*100))
        print(f'Time on train set after re-parameterization on GPU took: {(stop-start)/60} minutes')
    
    if validation: 
        start = time.time()
        preds = model.evaluate(val_ds, verbose=0)
        stop = time.time()
        print()
        print('Secondary Architecture')
        print('Loss and Train Accuracy after switching to deploy mode:', end= '\n')
        print('Validation Loss = {:.5f}'.format(preds[0]))
        print('Validation Accuracy = {:.2f}%'.format(preds[1]*100))
        print(f'Time on validation set with residual connections on GPU took: {(stop-start)/60} minutes')


        if test:
            start = time.time()
            preds = model.evaluate(test_ds, verbose=0)
            stop = time.time()
            print()
            print('Secondary Architecture')
            print('Loss and Test Accuracy after switching to deploy mode:', end= '\n')
            print('Test Loss = {:.5f}'.format(preds[0]))
            print('Test Accuracy = {:.2f}%'.format(preds[1]*100))
            print(f'Time on test set after re-parameterization on GPU took: {(stop-start)/60} minutes')


In [39]:
evaluate_model(train=True, validation=True, test=True, verbose=0)

Evaluating with primary architechture

Primary Architecture
Train Loss = 0.11144
Train Accuracy = 96.45%
Time on train set with residual connections on GPU took: 0.26534525156021116 minutes

Primary Architecture
Validation Loss = 0.22594
Validation Accuracy = 93.48%
Time on validation set with residual connections on GPU took: 0.06800949970881144 minutes

Primary Architecture
Test Loss = 0.23904
Test Accuracy = 93.64%
test time with residual connections on GPU took: 0.08110247850418091 minutes
Evaluating with secondary architechture/n
Secondary Architecture
Loss and Train Accuracy after switching to deploy mode:
Loss = 0.11144
Train Accuracy = 96.45%
Time on train set after re-parameterization on GPU took: 0.26372499465942384 minutes

Secondary Architecture
Loss and Train Accuracy after switching to deploy mode:
Validation Loss = 0.22594
Validation Accuracy = 93.48%
Time on validation set with residual connections on GPU took: 0.06834835608800252 minutes

Secondary Architecture
Loss an

In [ ]:
def load_history_df(path):
    return pd.read_csv(path)

In [41]:
model_name = "repvgg-7-layers"
repvgg_model_param = RepVGG(stage_inplanes= [64, 64, 128, 128, 128], num_blocks=[1, 2, 2, 1], in_channels=1, num_classes=3, name= model_name,
                 deploy=False, use_checkpoint=False)

RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7efe6adaac90>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7efe6ad0ac10>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7efe6ad1afd0>
RepVGG Block, identity =  None


In [43]:
repvgg_model_param.build((None,224,224,1))
repvgg_model_param.summary()

Model: "repvgg-7-layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rep_vgg_block_21 (RepVGGBloc multiple                  1152      
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
rep_vgg_block_22 (RepVGGBloc multiple                  41728     
_________________________________________________________________
rep_vgg_block_23 (RepVGGBloc multiple                  82944     
_________________________________________________________________
rep_vgg_block_24 (RepVGGBloc multiple                  165376    
_________________________________________________________________
rep_vgg_block_25 (RepVGGBloc multiple                  164864    
_________________________________________________________________
rep_vgg_block_26 (RepVGGBloc multiple              

In [44]:
model_name = "repvgg-7-layers"
repvgg_model_param_deploy = RepVGG(stage_inplanes= [64, 64, 128, 128, 128], num_blocks=[1, 2, 2, 1], in_channels=1, num_classes=3, name= model_name,
                 deploy=True, use_checkpoint=False)

In [45]:
repvgg_model_param_deploy.build((None,224,224,1))
repvgg_model_param_deploy.summary()

Model: "repvgg-7-layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rep_vgg_block_28 (RepVGGBloc multiple                  640       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
rep_vgg_block_29 (RepVGGBloc multiple                  36928     
_________________________________________________________________
rep_vgg_block_30 (RepVGGBloc multiple                  73856     
_________________________________________________________________
rep_vgg_block_31 (RepVGGBloc multiple                  147584    
_________________________________________________________________
rep_vgg_block_32 (RepVGGBloc multiple                  147584    
_________________________________________________________________
rep_vgg_block_33 (RepVGGBloc multiple              

In [48]:
702147 / 782979

0.8967635147302802

In [54]:
path = "/kaggle/working/resized_dataset"
train_ds_resized, val_ds_resized, test_ds_resized = prepare_dataset(path, size, color_mode, batch_size)

Found 4575 files belonging to 3 classes.
Using 2928 files for training.
Found 4575 files belonging to 3 classes.
Using 1647 files for validation.


In [55]:
train_ds_resized = train_ds_resized.cache()
train_ds_resized = train_ds_resized.prefetch(tf.data.experimental.AUTOTUNE)
val_ds_resized = val_ds_resized.cache()
val_ds_resized = val_ds_resized.prefetch(tf.data.experimental.AUTOTUNE)
test_ds_resized = test_ds_resized.cache()
test_ds_resized = test_ds_resized.prefetch(tf.data.experimental.AUTOTUNE)

In [57]:
# ###run for the first time to avoid 'Cleanup called...' message during the training
# for x , y in train_ds_resized:
#     pass

# for x , y in val_ds_resized:
#     pass

# for x , y in test_ds_resized:
#     pass

In [58]:
model_name = "repvgg-7-layers-resized"
repvgg_model_resized = RepVGG(stage_inplanes= [64, 64, 128, 128, 128], num_blocks=[1, 2, 2, 1], in_channels=1, num_classes=3, name= model_name,
                 deploy=False, use_checkpoint=False)
repvgg_history_resized = train_model(model= repvgg_model_resized, model_name= model_name, learning_rate= 0.0001,epochs= 60)

RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f031adeed50>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f031b0e6390>
RepVGG Block, identity =  None
RepVGG Block, identity =  <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f031afd5150>
RepVGG Block, identity =  None
Epoch 1/60
92/92 [==============================] - 64s 667ms/step - loss: 0.6131 - accuracy: 0.7749 - precision: 0.8126 - recall: 0.6899 - auc_2: 0.9066 - val_loss: 1.6536 - val_accuracy: 0.3395 - val_precision: 0.3395 - val_recall: 0.3395 - val_auc_2: 0.4927
Epoch 2/60
92/92 [==============================] - 61s 659ms/step - loss: 0.4576 - accuracy: 0.8449 - precision: 0.8642 - recall: 0.8258 - auc_2: 0.9466 - val_loss: 2.2836 - val_accuracy: 0.3395 - val_precision: 0.3395 - val_recall: 0.3395 - val_auc_2: 0.50

In [59]:
repvgg_model_resized.summary()

Model: "repvgg-7-layers-resized"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rep_vgg_block_35 (RepVGGBloc multiple                  1152      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
rep_vgg_block_36 (RepVGGBloc multiple                  41728     
_________________________________________________________________
rep_vgg_block_37 (RepVGGBloc multiple                  82944     
_________________________________________________________________
rep_vgg_block_38 (RepVGGBloc multiple                  165376    
_________________________________________________________________
rep_vgg_block_39 (RepVGGBloc multiple                  164864    
_________________________________________________________________
rep_vgg_block_40 (RepVGGBloc multiple      

In [60]:
repvgg_history_df_resized = pd.DataFrame(repvgg_history_resized.history)
repvgg_history_df_resized.to_csv('/kaggle/working/repvgg-7-layers-history_resized.csv')

In [61]:
evaluate_model(model= repvgg_model_resized, train=True, validation=True, test=True, verbose=0)

Evaluating with primary architechture

Primary Architecture
Train Loss = 0.17882
Train Accuracy = 94.64%
Time on train set with residual connections on GPU took: 0.2654226581255595 minutes

Primary Architecture
Validation Loss = 0.27268
Validation Accuracy = 92.01%
Time on validation set with residual connections on GPU took: 0.06864155530929565 minutes

Primary Architecture
Test Loss = 0.31574
Test Accuracy = 90.85%
test time with residual connections on GPU took: 0.08140837748845418 minutes
Evaluating with secondary architechture/n
Secondary Architecture
Loss and Train Accuracy after switching to deploy mode:
Loss = 0.17882
Train Accuracy = 94.64%
Time on train set after re-parameterization on GPU took: 0.2649903694788615 minutes

Secondary Architecture
Loss and Train Accuracy after switching to deploy mode:
Validation Loss = 0.27268
Validation Accuracy = 92.01%
Time on validation set with residual connections on GPU took: 0.06832919915517172 minutes

Secondary Architecture
Loss and 